In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import torch
from tqdm import tqdm

from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.tensorboard.writer import SummaryWriter

from sklearn.model_selection import train_test_split

from common import data
from common.training import training_loop, get_predictions

from algos import deep_fib

from common.models import resnet, deeplab
from common import metrics


# Hyperparameters

In [ ]:
log_dir = "./trained/deep_fib"

horizon = 1024
stride = 512

batch_size = 32

num_epochs = 10
learning_rate = 1e-3
weight_decay = 1e-2

n_masks = 25
loss_type = "mse"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


# Datasets

In [ ]:
paths = data.get_dataset_paths("../data")
train, test = train_test_split(paths, test_size=0.1, random_state=42)


In [ ]:
m_data_train = data.Marconi100Dataset(train, scaling=data.Scaling.MINMAX)
m_data_test = data.Marconi100Dataset(test, scaling=data.Scaling.MINMAX)


In [ ]:
dataset_train = data.UnfoldedDataset(m_data_train, horizon=horizon, stride=stride)
dataset_test = data.UnfoldedDataset(m_data_test, horizon=horizon, stride=stride)

masks = deep_fib.get_masks(horizon, n_masks).float()

print(len(dataset_train), len(dataset_test), masks.size())


In [ ]:
train_loader = DataLoader(
    dataset_train,
    batch_size,
    shuffle=True,
)
test_loader = DataLoader(
    dataset_test,
    batch_size,
    shuffle=False,
)
print(len(train_loader), len(test_loader))


# Model

In [ ]:
model = deeplab.DeepLabNet(
    resnet.ResNetFeatures(
        resnet.Bottleneck,
        resnet.RESNET50_LAYERS,
        return_layers=[resnet.LAYER_1, resnet.LAYER_4],
        replace_stride_with_dilation=[False, True, True],
        num_features=data.NUM_FEATURES,
    ),
    backbone_channels=[256, 2048],
    out_feats=data.NUM_FEATURES,
).float()

optim = Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
lr_sched = CosineAnnealingLR(optim, num_epochs)
engine = deep_fib.DeepFIBEngine(
    model=model,
    device=device,
    optimizer=optim,
    lr_scheduler=lr_sched,
    masks=masks,
    loss_type=loss_type,
)


# Training

In [ ]:
with SummaryWriter(log_dir) as writer:
    training_loop(
        engine=engine,
        num_epochs=num_epochs,
        train_dataloader=train_loader,
        test_dataloader=test_loader,
        writer=writer,
        save_path=log_dir + "/models",
    )


# Evaluating

In [ ]:
# Cost model
cmodel = metrics.default_cmodel()


In [ ]:
all_errors, all_labels = get_predictions(engine.predict, test_loader)


In [ ]:
cmodel.fit(all_errors, all_labels).optimize()


In [ ]:
metrics.plot_cost(cmodel)


In [ ]:
metrics.plot_errors_curve(
    cmodel.false_positives, cmodel.false_negatives, cmodel.thresholds, figsize=(15, 5)
)


In [ ]:
metrics.plot_precision_recall_f1_curve(
    cmodel.precision, cmodel.recall, cmodel.f1_score, cmodel.thresholds
)
